<a href="https://colab.research.google.com/github/kacioidarlan/FakeNewsDetector/blob/main/AI_Fake_News_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install Google SDK
!pip install -q -U google-generativeai

In [3]:
# Import the Python SDK
import google.generativeai as genai
from google.colab import userdata

gemini_api_key = userdata.get('gemini_api_key')
genai.configure(api_key=gemini_api_key)

In [4]:
# Preparando configurações do modelo, mas senão for passado a API tem um padrão para todos os parâmetros.

generation_config = {
    'candidate_count': 1,
    'temperature': 0.5,
}

safety_settings = {
    'HARASSMENT': 'BLOCK_NONE',
    'HATE': 'BLOCK_NONE',
    'SEXUAL': 'BLOCK_NONE',
    'DANGEROUS': 'BLOCK_NONE',
}

In [19]:
# Inicializando o modelo
model =genai.GenerativeModel(model_name='gemini-1.5-pro-latest',
                             generation_config=generation_config,
                             safety_settings=safety_settings)

In [22]:
# Criaçao do Agente de Verificação de Fatos
# Possibilidades futuras:
# adicionar pesquisa do Google para agregar e para casos específicos,
# adicionar documentos para usar como RAG, por exemplo artigos científicos sovre determinada especialidade, por exemplo as vanicas de CODIV 19

class AgenteVerificacaoFatos:
    def __init__(self, model):
        self.model = model

    def verificar_fatos(self, noticia):
        prompt = f"""
        Você é um especialista em verificação de fatos.
        Analise as informações na seguinte notícia e verifique sua veracidade consultando fontes confiáveis:

        "{noticia}"

        Responda com:
        - "VERDADEIRO" se a notícia for factualmente correta.
        - "FALSO" se a notícia contiver informações falsas.
        - "INCONCLUSIVO" se não for possível determinar a veracidade.

        Apresente as evidências encontradas e os links para as fontes que sustentam sua conclusão.
        """
        response = self.model.generate_content(prompt)
        return response.text

In [14]:
# Criação do Agente de Deteccao Tendencias normalmente utilizadas em fake news
class AgenteDeteccaoTendencias:
    def __init__(self, model):
        self.model = model

    def detectar_tendencias(self, noticia):
        prompt = f"""
        Você é um especialista em identificar tendências de desinformação.
        Analise a seguinte notícia:

        "{noticia}"

        Indique se a notícia apresenta características de:
        - Clickbait
        - Sensacionalismo
        - Teorias da conspiração
        - Propaganda política
        - Outras tendências comuns de fake news

        Apresente exemplos específicos da notícia que justifiquem sua análise.
        """
        response = self.model.generate_content(prompt)
        return response.text

In [15]:
# Criação do Agende de Análise de Sentimento
class AgenteAnaliseSentimento:
    def __init__(self, model):
        self.model = model

    def analisar_sentimento(self, noticia):
        prompt = f"""
        Você é um especialista em análise de sentimentos em textos.
        Analise a linguagem utilizada na seguinte notícia:

        "{noticia}"

        O tom da notícia é:
        - Neutro e objetivo
        - Positivo e otimista
        - Negativo e pessimista
        - Alarmista e sensacionalista
        - Outro (especifique)

        Justifique sua resposta com exemplos da notícia.
        """
        response = self.model.generate_content(prompt)
        return response.text

In [16]:
# Criação do Agente que irá fazer a conclusão
class AgenteAnaliseFinal:
    def analisar(self, respostas_agentes):
        prompt = f"""
        Você é um juiz responsável por determinar se uma notícia é confiável ou não.
        Analise os seguintes relatórios de especialistas:

        Verificação de Fatos:
        {respostas_agentes['verificacao']}

        Detecção de Tendências:
        {respostas_agentes['tendencias']}

        Análise de Sentimento:
        {respostas_agentes['sentimento']}

        Com base nessas informações, a notícia é:
        - Confiável
        - Suspeita
        - Falsa
        - Inconclusivo

        Justifique sua decisão.
        """
        response = model.generate_content(prompt)
        classificacao = response.text
        return classificacao

In [17]:
# Instanciar os agentes
agente_verificacao = AgenteVerificacaoFatos(model)
agente_tendencias = AgenteDeteccaoTendencias(model)
agente_sentimento = AgenteAnaliseSentimento(model)
agente_final = AgenteAnaliseFinal()

**Notícia de teste:**
Mensagem circula nas redes sociais
“ATENÇÃO!!! Assim que a água voltar não tome, não dê para as crianças, não use para cozinhar, pois infelizmente o Guaíba está cheio de corpos e restos cadavéricos, é indicado que espere no mínimo de 15 a 20 dias para fazer o consumo da água, mesmo a água passando pelo tratamento ocorre um grande risco de contaminação. REPASSEM A MENSAGEM!!!”


In [21]:
# Orquestração do Fluxo
noticia = input("Digite a notícia: ")

resposta_verificacao = agente_verificacao.verificar_fatos(noticia)
resposta_tendencias = agente_tendencias.detectar_tendencias(noticia)
resposta_sentimento = agente_sentimento.analisar_sentimento(noticia)

respostas_agentes = {
    "verificacao": resposta_verificacao,
    "tendencias": resposta_tendencias,
    "sentimento": resposta_sentimento,
}

classificacao_final = agente_final.analisar(respostas_agentes)

print(f"Classificação Final: {classificacao_final}")

Digite a notícia: Mensagem circula nas redes sociais “ATENÇÃO!!! Assim que a água voltar não tome, não dê para as crianças, não use para cozinhar, pois infelizmente o Guaíba está cheio de corpos e restos cadavéricos, é indicado que espere no mínimo de 15 a 20 dias para fazer o consumo da água, mesmo a água passando pelo tratamento ocorre um grande risco de contaminação. REPASSEM A MENSAGEM!!!”
Classificação Final: Com base nas informações fornecidas, a notícia é **FALSA**. 

**Justificativa:**

1. **Verificação de fatos:** Diversas fontes confiáveis, incluindo o site oficial da CORSAN, o Ministério da Saúde e a OMS, afirmam que a água tratada é segura para consumo humano. A alegação de contaminação por corpos e restos cadavéricos no Guaíba não possui qualquer evidência ou fonte confiável.

2. **Detecção de tendências:** A notícia apresenta diversas características de fake news, incluindo:
    * Sensacionalismo e linguagem alarmista para gerar medo e pânico.
    * Teorias da conspiração